## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-27-19-05-12 +0000,nypost,Amazon tribe facing extinction welcomes new me...,https://nypost.com/2026/02/27/world-news/amazo...
1,2026-02-27-19-04-07 +0000,nyt,EEOC Says Agencies Can Restrict Bathroom Use b...,https://www.nytimes.com/2026/02/27/us/politics...
2,2026-02-27-19-03-10 +0000,bbc,"Tram derails in Milan, leaving two dead and do...",https://www.bbc.com/news/articles/c5y599degnqo...
3,2026-02-27-19-01-55 +0000,nypost,30 more people charged in Minnesota church sto...,https://nypost.com/2026/02/27/us-news/30-more-...
4,2026-02-27-19-00-00 +0000,wsj,Our Parents Marched in Iran’s 1979 Revolution....,https://www.wsj.com/world/middle-east/iran-pro...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2610/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
115,trump,49
35,iran,23
224,war,17
481,ai,17
86,epstein,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
28,2026-02-27-18-25-13 +0000,nypost,Trump makes sobering admission about Iran talk...,https://nypost.com/2026/02/27/us-news/trump-no...,144
289,2026-02-27-00-03-07 +0000,nypost,House Dems to force vote to block Trump from w...,https://nypost.com/2026/02/26/us-news/house-de...,113
34,2026-02-27-18-03-16 +0000,nypost,Forcing Bill Clinton to testify will backfire ...,https://nypost.com/2026/02/27/us-news/forcing-...,112
284,2026-02-27-00-25-42 +0000,nypost,Trump moves more military might to the Middle ...,https://nypost.com/2026/02/26/us-news/trump-mo...,111
333,2026-02-26-22-21-29 +0000,wapo,"U.S., Iran complete round of talks as Trump we...",https://www.washingtonpost.com/world/2026/02/2...,104


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
28,144,2026-02-27-18-25-13 +0000,nypost,Trump makes sobering admission about Iran talk...,https://nypost.com/2026/02/27/us-news/trump-no...
276,80,2026-02-27-00-53-44 +0000,nypost,Paramount Skydance victory in Warner Bros. Dis...,https://nypost.com/2026/02/26/media/paramount-...
88,73,2026-02-27-16-15-50 +0000,nypost,Nancy Mace claims Hillary Clinton was ‘scream...,https://nypost.com/2026/02/27/us-news/nancy-ma...
191,50,2026-02-27-10-30-00 +0000,wsj,"Neither theory, history nor the latest data su...",https://www.wsj.com/economy/jobs/tech-has-neve...
138,48,2026-02-27-13-19-18 +0000,bbc,Pakistan defence minister says country in 'ope...,https://www.bbc.com/news/articles/cvgjd7pv4y4o...
360,46,2026-02-26-21-28-00 +0000,wsj,Oil lobbyists have made the case to Trump offi...,https://www.wsj.com/politics/policy/big-oil-is...
43,46,2026-02-27-17-39-26 +0000,nypost,$1M Guthrie family reward can be paid in cash ...,https://nypost.com/2026/02/27/us-news/savannah...
228,46,2026-02-27-06-07-00 +0000,wsj,The Pentagon said it shot down a “seemingly th...,https://www.wsj.com/politics/national-security...
373,43,2026-02-26-20-50-58 +0000,nypost,Meta tried to block lawyers from asking Mark Z...,https://nypost.com/2026/02/26/business/meta-tr...
259,43,2026-02-27-02-00-00 +0000,wsj,Republicans are worried that a MAGA vs. MAGA f...,https://www.wsj.com/politics/elections/the-spe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
